In [1]:
from sqlalchemy.engine import URL
from sqlalchemy import Column, Integer, String, DateTime, Text, ForeignKey, Uuid, SmallInteger, DECIMAL
from sqlalchemy.orm import declarative_base
from datetime import datetime
from sqlalchemy.orm import sessionmaker
from home_security.data.postgresql.database import get_session, create_tables, seed_db
from home_security.data.postgresql.models import Household, Camera, ImageSet, IntrusionImage, Detection
from home_security.data.storage import StorageDriver
from PIL import Image
from typing import List
from home_security.data.minio.minio_client import Minio
import uuid
from home_security.data.postgresql.database import Base, engine
Base.metadata.drop_all(engine) # this deletes all tables !!!
minio = Minio(endpoint="localhost:9000", access_key='homesecurity', secret_key='homesecuritybyjohn')
session = get_session()
create_tables()
seed_db()

storage_driver = StorageDriver(minio=minio, session=session)

In [2]:
filename = "D01-1_20240605104950.jpg"
image = Image.open(f'../images/new_images/denning/{filename}')

In [3]:
storage_driver.add_image_set(images=[image, image], camera_id='D02', datetime_str='20250223144502')

In [4]:
image_sets = storage_driver.get_uninferenced_image_sets()

In [5]:
image_sets

In [13]:
# initialise model
from home_security.model import load_model
from home_security.inference import inference
model, processor = load_model()

for image_set in image_sets:
    dets_to_register = []
    images = storage_driver.get_images_from_set(image_set)
    for idx, image in enumerate(images):
        detections = inference(
            model=model, processor=processor, image=image, min_score=0.001
        )
        for det in detections:
            dets_to_register.append(Detection(image_id=image_set.intrusion_images[idx].id, label=det['label'], confidence=det['score'], x_min=det['box'][0], y_min=det['box'][1], x_max=det['box'][2], y_max=det['box'][3]))
    session.add_all(dets_to_register)
    session.commit()
    image_set.inferenced = True


In [17]:
image_sets[0].intrusion_images[1].detections